In [1]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
data = pd.read_csv("../data_examples/csv/raw_detections.csv", index_col=False)

In [5]:
def apply_correction_dfs(data):
    
    # create copy of dataframe
    df = data.copy()
    df.set_index("candid", inplace=True)
    
    # map isdiffpos
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., '1': 1, '0': -1, 'f': -1.})

    # apply correction where distnr < 1.4
    df["corrected"] = df["distnr"] < 1.4

    # correct
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos, oid=df.objectId.iloc[0], candid=x.name) if x["corrected"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    try:
        df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    except:
        display(correction_results)
        print(correction_results)
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    corr_magstats = df.loc[df.index.min()]["corrected"]
    mask = ((df["corrected"] == False) & (df.isdiffpos == -1)) | (corr_magstats & (df["corrected"] == False)) | ((corr_magstats == False) & df["corrected"])
    df["dubious"] = mask
    
    return df

In [8]:
a = data.groupby(["objectId", "fid"]).apply(apply_correction_dfs)

In [11]:
b = data.groupby(["objectId", "fid"]).apply(apply_correction_df)

In [15]:
a[a.dubious == True]

,,,objectId,fid,jd,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,tblid,...,maggaiabright,exptime,drb,drbversion,candid_parent,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious
objectId,fid,candid,,,,,,,,,,,,,,,,,,,,,
ZTF18aaiscil,1,1032528244615010002,ZTF18aaiscil,1,2.458787e+06,1032528244615,18.4697,/ztf/archive/sci/2019/1030/528241/ztf_20191030...,Kulkarni,1,-1.0,2,...,NaN,NaN,NaN,NaN,1.058557e+18,False,NaN,NaN,NaN,True


In [21]:
is_dubious(True, True, False)

1

In [ ]:
# test c2
data.at[180, "isdiffpos"] = 'f'

In [ ]:
# test c3
data.at[150, "distnr"] = 0.1

In [ ]:
print(data.objectId.unique(), data.fid.unique())

In [ ]:
pd.options.display.max_rows = 999
display(data[["objectId", "fid", "magpsf", "distnr", "chinr", "sharpnr"]])

In [ ]:
def apply_correction_df(data):
    
    # apply correction where possible, save corrected column
    df = data.copy()
    #display(df.head(1))
    #print(df.objectId.unique(), df.fid.unique())
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    df["corr"] = df["distnr"] < 1.4
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos) if x["corr"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    
    # check for consistency
    idxmin = df.candid.idxmin()
    df["corr_magstats"] = df.loc[idxmin]["corr"]
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    mask = ((df["corr"] == False) & (df.isdiffpos == -1)) | (df.corr_magstats & (df["corr"] == False)) | ((df.corr_magstats == False) & df["corr"])
    df["dubious"] = mask
    
    # stellar_object
    idxmin = df.candid.idxmin()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar
    df["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    df["stellar_magstats"] = (nearZTF & stellarZTF)
    
    return df

In [ ]:
def apply_magstats(df):
    response = {}
    idxmin = df.candid.idxmin()
    idxmax = df.candid.idxmax()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar 
    response["ndet"] = len(df)
    response["rfid"] = df.loc[idxmin].rfid # ?
    response["nrfid"] = len(df.rfid.dropna().unique())
    response["magnr"] = df.loc[idxmin].magnr # ?
    response["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    response["stellar_magstats"] = (nearZTF & stellarZTF)
    response["magap_mean"] = df.magap.mean()
    response["magap_median"] = df.magap.median()
    response["magap_max"] = df.magap.max()
    response["magap_min"] = df.magap.min()
    response["magap_fisrt"] = df.loc[idxmin].magap
    response["magap_last"] = df.loc[idxmax].magap
    response["first_mjd"] = df.loc[idxmin].jd - 2400000.5
    response["last_mjd"] = df.loc[idxmax].jd - 2400000.5
    return pd.Series(response)

In [ ]:
dflarge = data.groupby(["objectId", "fid"]).apply(apply_correction_df)

In [ ]:
dflarge[["objectId", "fid", "distnr", "distpsnr1", "sgscore1", "chinr", "sharpnr", "corr", "corr_magstats", "dubious", "stellar_object", "stellar_magstats"]]

In [ ]:
groups = data.groupby(["objectId", "fid"])

In [ ]:
magstats = groups.apply(apply_magstats)
display(magstats)